# GEO-AI Challenge for Cropland Mapping by ITU


Authenticate and initialize Earth Engine.

**You will need an Earth Engine account. It can be requested having a Google account at https://earthengine.google.com/**

You will be prompted to enter your credentials after running the lines below

In [1]:
# Basic & Built-in
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from datetime import datetime, timedelta

# Google Earth Engine
import ee

In [2]:
# Set seed for reproducability
SEED = 2023
random.seed(SEED)
np.random.seed(SEED)

## Paths

In [3]:
# Load the files - make sure to have the correct data path

####### Please sepcify where you are saving the data - same path will be used to save any outputs ###########
data_dr = './data'

# Train data
tr_sud = pd.read_csv(f'{data_dr}/train_sudan.csv')
tr_irn = pd.read_csv(f'{data_dr}/train_iran.csv')
tr_afg = pd.read_csv(f'{data_dr}/train_afg.csv')

# Test data
te_sud = pd.read_csv(f'{data_dr}/test_sudan.csv')
te_irn = pd.read_csv(f'{data_dr}/test_iran.csv')
te_afg = pd.read_csv(f'{data_dr}/test_afg.csv')


# Submission file
sample_submission = pd.read_csv(f'{data_dr}/SampleSubmission.csv')

In [4]:
# Get authetication token and sign in to Google Earth Engine
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AfJohXl2EHmoui1yJmZg6xpHVCc1pJp79J0jEyNc2DLlgQ6PYdKQQjyRRkI



Successfully saved authorization token.


## Extract the datasets from Google Earth Engine

### Sentinel-2
Load Sentinel-2 imagery from Earth Engine and select the bands:

          [B2, B3, B4, B5 ,B6, B7, B8A, B11, B12, B8]

the monthly mean value extractd fot the entire time interval, as following:

1. Afghanistan: extracted the monthly mean value of April 2022. Also, monthly mean values for the past 3 months (Jan 2022 ~ March 2022) has been extracted and added to the database for a near real time assessment.
2. Iran: July 2019 ~ June 2020.
3. Sudan: July 2019 ~ June 2020.

In [5]:
%%time
BANDS = ['B2', 'B3', 'B4','B5','B6','B7','B8A','B11','B12', 'B8']

### Read the CSV table
tr_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(tr_afg['Lon'], tr_afg['Lat'])]
te_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(te_afg['Lon'], te_afg['Lat'])]



start_date = datetime.strptime("2022-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2022-04-30", "%Y-%m-%d")


t = 1
while start_date < end_date:
    suffix = f'_t{t}'
    BANDS_suffixes =[band + suffix for band in BANDS]

    tr_afg[BANDS_suffixes] = None
    te_afg[BANDS_suffixes] = None

    current_date = start_date
    next_date = start_date + timedelta(days=30)
    print('############', current_date, ' to ', next_date)
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))


    #Extract train data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(tr_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(tr_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        tr_afg.loc[i, BANDS_suffixes] = values
        i+=1


     #Extract test data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(te_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(te_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        te_afg.loc[i, BANDS_suffixes] = values
        i+=1

    start_date = next_date
    t+=1

############ 2022-01-01 00:00:00  to  2022-01-31 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1229.93it/s]


############ 2022-01-31 00:00:00  to  2022-03-02 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1410.22it/s]


############ 2022-03-02 00:00:00  to  2022-04-01 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1481.05it/s]


############ 2022-04-01 00:00:00  to  2022-05-01 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1051.15it/s]

CPU times: user 3.54 s, sys: 68.9 ms, total: 3.61 s
Wall time: 38.1 s


In [6]:
print(te_afg.shape)
print(tr_afg.shape)

(500, 45)
(500, 46)


In [ ]:
BANDS = ['B2', 'B3', 'B4','B5','B6','B7','B8A','B11','B12', 'B8']

# Read the points geometries from the CSV table
tr_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(tr_sud['Lon'], tr_sud['Lat'])]
te_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(te_sud['Lon'], te_sud['Lat'])]



start_date = datetime.strptime("2019-07-01", "%Y-%m-%d")
end_date = datetime.strptime("2020-06-30", "%Y-%m-%d")


t = 1
while start_date < end_date:
    suffix = f'_t{t}'
    BANDS_suffixes =[band + suffix for band in BANDS]

    tr_sud[BANDS_suffixes] = None
    te_sud[BANDS_suffixes] = None

    current_date = start_date
    next_date = start_date + timedelta(days=30)
    print('############', current_date, ' to ', next_date)
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))


    #Extract train data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(tr_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(tr_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        tr_sud.loc[i, BANDS_suffixes] = values
        i+=1


     #Extract test data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(te_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(te_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        te_sud.loc[i, BANDS_suffixes] = values
        i+=1

    start_date = next_date
    t+=1

############ 2019-07-01 00:00:00  to  2019-07-31 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1164.45it/s]


############ 2019-07-31 00:00:00  to  2019-08-30 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1526.27it/s]


############ 2019-08-30 00:00:00  to  2019-09-29 00:00:00


...

100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1188.32it/s]
/tmp/ipykernel_2757002/3033152139.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_sud[BANDS_suffixes] = None
/tmp/ipykernel_2757002/3033152139.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_sud[BANDS_suffixes] = None
/tmp/ipykernel_2757002/3033152139.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at

############ 2020-05-26 00:00:00  to  2020-06-25 00:00:00


100%|████████████████████████████████████████| 500/500 [00:00<00:00, 648.43it/s]
/tmp/ipykernel_2757002/3033152139.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_sud[BANDS_suffixes] = None
/tmp/ipykernel_2757002/3033152139.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_sud[BANDS_suffixes] = None
/tmp/ipykernel_2757002/3033152139.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at

############ 2020-06-25 00:00:00  to  2020-07-25 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1229.36it/s]


In [ ]:
print(tr_sud.shape)
print(te_sud.shape)

(500, 136)
(500, 135)


In [9]:
# Read the points geometries from the CSV table
tr_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(tr_irn['Lon'], tr_irn['Lat'])]
te_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(te_irn['Lon'], te_irn['Lat'])]


start_date = datetime.strptime("2019-07-01", "%Y-%m-%d")
end_date = datetime.strptime("2020-06-30", "%Y-%m-%d")


t = 1
while start_date < end_date:
    suffix = f'_t{t}'
    BANDS_suffixes =[band + suffix for band in BANDS]

    tr_irn[BANDS_suffixes] = None
    te_irn[BANDS_suffixes] = None

    current_date = start_date
    next_date = start_date + timedelta(days=30)
    print('############', current_date, ' to ', next_date)
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))


    #Extract train data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(tr_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(tr_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        tr_irn.loc[i, BANDS_suffixes] = values
        i+=1


     #Extract test data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(te_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(te_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        te_irn.loc[i, BANDS_suffixes] = values
        i+=1

    start_date = next_date
    t+=1

############ 2019-07-01 00:00:00  to  2019-07-31 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1551.43it/s]


############ 2019-07-31 00:00:00  to  2019-08-30 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1517.93it/s]


############ 2019-08-30 00:00:00  to  2019-09-29 00:00:00


...

100%|████████████████████████████████████████| 500/500 [00:01<00:00, 486.78it/s]
/tmp/ipykernel_2757002/1309579946.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_irn[BANDS_suffixes] = None
/tmp/ipykernel_2757002/1309579946.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_irn[BANDS_suffixes] = None
/tmp/ipykernel_2757002/1309579946.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at

############ 2020-05-26 00:00:00  to  2020-06-25 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1323.15it/s]
/tmp/ipykernel_2757002/1309579946.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_irn[BANDS_suffixes] = None
/tmp/ipykernel_2757002/1309579946.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_irn[BANDS_suffixes] = None
/tmp/ipykernel_2757002/1309579946.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at

############ 2020-06-25 00:00:00  to  2020-07-25 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1107.72it/s]


In [10]:
print(tr_irn.shape)
print(te_irn.shape)

(500, 136)
(500, 135)


In [11]:
tr_afg.to_csv(f'tr_afg_s2.csv', index=False )
te_afg.to_csv(f'te_afg_s2.csv', index=False)

tr_sud.to_csv(f'tr_sud_s2.csv', index=False)
te_sud.to_csv(f'te_sud_s2.csv', index=False)

tr_irn.to_csv(f'tr_irn_s2.csv', index=False)
te_irn.to_csv(f'te_irn_s2.csv', index=False)

### Soil Moisture
Load The SMAP Level-4 (L4) Soil Moisture product from Earth Engine and select the bands:

         'sm_surface', 'sm_rootzone',
         'sm_surface_wetness','sm_rootzone_wetness',
          'surface_temp', 'land_evapotranspiration_flux',
         'vegetation_greenness_fraction', 'leaf_area_index'

the monthly mean value extractd fot the entire time interval, as following:

1. Afghanistan: extracted the monthly mean value of April 2022. Also, monthly mean values for the past 3 months (Jan 2022 ~ March 2022) has been extracted and added to the database for a near real time assessment.
2. Iran: July 2019 ~ June 2020.
3. Sudan: July 2019 ~ June 2020.

In [12]:
# Train data
tr_sud = pd.read_csv(f'{data_dr}/train_sudan.csv')
tr_irn = pd.read_csv(f'{data_dr}/train_iran.csv')
tr_afg = pd.read_csv(f'{data_dr}/train_afg.csv')

# Test data
te_sud = pd.read_csv(f'{data_dr}/test_sudan.csv')
te_irn = pd.read_csv(f'{data_dr}/test_iran.csv')
te_afg = pd.read_csv(f'{data_dr}/test_afg.csv')

In [13]:
%%time

BANDS = ['sm_surface', 'sm_rootzone','sm_surface_wetness','sm_rootzone_wetness',
         'surface_temp', 'land_evapotranspiration_flux',
         'vegetation_greenness_fraction', 'leaf_area_index' ]

# Read the points geometries from the CSV table
tr_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(tr_afg['Lon'], tr_afg['Lat'])]
te_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(te_afg['Lon'], te_afg['Lat'])]


start_date = datetime.strptime("2022-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2022-04-30", "%Y-%m-%d")


t = 1
while start_date < end_date:
    suffix = f'_t{t}'
    BANDS_suffixes =[band + suffix for band in BANDS]

    tr_afg[BANDS_suffixes] = None
    te_afg[BANDS_suffixes] = None

    current_date = start_date
    next_date = start_date + timedelta(days=30)
    print('############', current_date, ' to ', next_date)
    collection = (ee.ImageCollection('NASA/SMAP/SPL4SMGP/007').filterDate(current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))


    #Extract train data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(tr_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(tr_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        tr_afg.loc[i, BANDS_suffixes] = values
        i+=1


     #Extract test data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(te_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(te_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        te_afg.loc[i, BANDS_suffixes] = values
        i+=1

    start_date = next_date
    t+=1


############ 2022-01-01 00:00:00  to  2022-01-31 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1613.53it/s]


############ 2022-01-31 00:00:00  to  2022-03-02 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1439.73it/s]


############ 2022-03-02 00:00:00  to  2022-04-01 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1572.62it/s]


############ 2022-04-01 00:00:00  to  2022-05-01 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1459.87it/s]

CPU times: user 3.19 s, sys: 54 ms, total: 3.24 s
Wall time: 7min 24s


In [14]:
# Read the points geometries from the CSV table
tr_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(tr_sud['Lon'], tr_sud['Lat'])]
te_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(te_sud['Lon'], te_sud['Lat'])]



start_date = datetime.strptime("2019-07-01", "%Y-%m-%d")
end_date = datetime.strptime("2020-06-30", "%Y-%m-%d")


t = 1
while start_date < end_date:
    suffix = f'_t{t}'
    BANDS_suffixes =[band + suffix for band in BANDS]

    tr_sud[BANDS_suffixes] = None
    te_sud[BANDS_suffixes] = None

    current_date = start_date
    next_date = start_date + timedelta(days=30)
    print('############', current_date, ' to ', next_date)
    collection = (ee.ImageCollection('NASA/SMAP/SPL4SMGP/007').filterDate(current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))


    #Extract train data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(tr_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(tr_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        tr_sud.loc[i, BANDS_suffixes] = values
        i+=1


     #Extract test data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(te_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(te_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        te_sud.loc[i, BANDS_suffixes] = values
        i+=1

    start_date = next_date
    t+=1

############ 2019-07-01 00:00:00  to  2019-07-31 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1647.67it/s]


############ 2019-07-31 00:00:00  to  2019-08-30 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1174.91it/s]


############ 2019-08-30 00:00:00  to  2019-09-29 00:00:00


...

100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1386.10it/s]


############ 2020-05-26 00:00:00  to  2020-06-25 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1442.20it/s]
/tmp/ipykernel_2757002/973793766.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_sud[BANDS_suffixes] = None
/tmp/ipykernel_2757002/973793766.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_sud[BANDS_suffixes] = None
/tmp/ipykernel_2757002/973793766.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at on

############ 2020-06-25 00:00:00  to  2020-07-25 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1314.28it/s]


In [15]:
%%time

# Read the points geometries from the CSV table
tr_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(tr_irn['Lon'], tr_irn['Lat'])]
te_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(te_irn['Lon'], te_irn['Lat'])]


start_date = datetime.strptime("2019-07-01", "%Y-%m-%d")
end_date = datetime.strptime("2020-06-30", "%Y-%m-%d")


t = 1
while start_date < end_date:
    suffix = f'_t{t}'
    BANDS_suffixes =[band + suffix for band in BANDS]

    tr_irn[BANDS_suffixes] = None
    te_irn[BANDS_suffixes] = None

    current_date = start_date
    next_date = start_date + timedelta(days=30)
    print('############', current_date, ' to ', next_date)
    collection = (ee.ImageCollection('NASA/SMAP/SPL4SMGP/007').filterDate(current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))


    #Extract train data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(tr_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(tr_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        tr_irn.loc[i, BANDS_suffixes] = values
        i+=1


     #Extract test data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(te_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(te_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        te_irn.loc[i, BANDS_suffixes] = values
        i+=1

    start_date = next_date
    t+=1

############ 2019-07-01 00:00:00  to  2019-07-31 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1405.02it/s]


############ 2019-07-31 00:00:00  to  2019-08-30 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1509.89it/s]


############ 2019-08-30 00:00:00  to  2019-09-29 00:00:00


...

############ 2020-05-26 00:00:00  to  2020-06-25 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1367.32it/s]
<timed exec>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:15: PerformanceWarning: 

############ 2020-06-25 00:00:00  to  2020-07-25 00:00:00


100%|████████████████████████████████████████| 500/500 [00:00<00:00, 531.50it/s]

CPU times: user 12.4 s, sys: 340 ms, total: 12.8 s
Wall time: 39min 51s


In [16]:
tr_afg.to_csv(f'tr_afg_smap.csv', index=False )
te_afg.to_csv(f'te_afg_smap.csv', index=False)

tr_sud.to_csv(f'tr_sud_smap.csv', index=False)
te_sud.to_csv(f'te_sud_smap.csv', index=False)

tr_irn.to_csv(f'tr_irn_smap.csv', index=False)
te_irn.to_csv(f'te_irn_smap.csv', index=False)

### Sentinel-1

Load Sentinel-1 imagery from Earth Engine and select the bands:

        [VV, VH]

the monthly mean value extractd fot the entire time interval, as following:

1. Afghanistan: extracted the monthly mean value of April 2022. Also, monthly mean values for the past 3 months (Jan 2022 ~ March 2022) has been extracted and added to the database for a near real time assessment.
2. Iran: July 2019 ~ June 2020.
3. Sudan: July 2019 ~ June 2020.

In [17]:
# Train data
tr_sud = pd.read_csv(f'{data_dr}/train_sudan.csv')
tr_irn = pd.read_csv(f'{data_dr}/train_iran.csv')
tr_afg = pd.read_csv(f'{data_dr}/train_afg.csv')

# Test data
te_sud = pd.read_csv(f'{data_dr}/test_sudan.csv')
te_irn = pd.read_csv(f'{data_dr}/test_iran.csv')
te_afg = pd.read_csv(f'{data_dr}/test_afg.csv')


In [18]:
%%time

BANDS = ['VV', 'VH']

# Read the points geometries from the CSV table
tr_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(tr_afg['Lon'], tr_afg['Lat'])]
te_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(te_afg['Lon'], te_afg['Lat'])]


start_date = datetime.strptime("2022-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2022-04-30", "%Y-%m-%d")

t = 1
while start_date < end_date:
    suffix = f'_t{t}'
    BANDS_suffixes =[band + suffix for band in BANDS]

    tr_afg[BANDS_suffixes] = None
    te_afg[BANDS_suffixes] = None

    current_date = start_date
    next_date = start_date + timedelta(days=30)
    print('############', current_date, ' to ', next_date)
    collection = (ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))


    #Extract train data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(tr_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(tr_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        tr_afg.loc[i, BANDS_suffixes] = values
        i+=1


     #Extract test data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(te_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(te_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        te_afg.loc[i, BANDS_suffixes] = values
        i+=1

    start_date = next_date
    t+=1

############ 2022-01-01 00:00:00  to  2022-01-31 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1941.40it/s]


############ 2022-01-31 00:00:00  to  2022-03-02 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 2020.44it/s]


############ 2022-03-02 00:00:00  to  2022-04-01 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 2099.20it/s]


############ 2022-04-01 00:00:00  to  2022-05-01 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1862.38it/s]

CPU times: user 2.56 s, sys: 44.4 ms, total: 2.6 s
Wall time: 30 s


In [19]:
# Read the points geometries from the CSV table
tr_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(tr_sud['Lon'], tr_sud['Lat'])]
te_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(te_sud['Lon'], te_sud['Lat'])]



start_date = datetime.strptime("2019-07-01", "%Y-%m-%d")
end_date = datetime.strptime("2020-06-30", "%Y-%m-%d")


t = 1
while start_date < end_date:
    suffix = f'_t{t}'
    BANDS_suffixes =[band + suffix for band in BANDS]

    tr_sud[BANDS_suffixes] = None
    te_sud[BANDS_suffixes] = None

    current_date = start_date
    next_date = start_date + timedelta(days=30)
    print('############', current_date, ' to ', next_date)
    collection = (ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))


    #Extract train data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(tr_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(tr_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        tr_sud.loc[i, BANDS_suffixes] = values
        i+=1


     #Extract test data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(te_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(te_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        te_sud.loc[i, BANDS_suffixes] = values
        i+=1

    start_date = next_date
    t+=1

############ 2019-07-01 00:00:00  to  2019-07-31 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1758.41it/s]


############ 2019-07-31 00:00:00  to  2019-08-30 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 2073.03it/s]


############ 2019-08-30 00:00:00  to  2019-09-29 00:00:00


...

100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1184.28it/s]


############ 2020-05-26 00:00:00  to  2020-06-25 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1977.67it/s]


############ 2020-06-25 00:00:00  to  2020-07-25 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1149.09it/s]


In [20]:
%%time

# Read the points geometries from the CSV table
tr_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(tr_irn['Lon'], tr_irn['Lat'])]
te_point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(te_irn['Lon'], te_irn['Lat'])]


start_date = datetime.strptime("2019-07-01", "%Y-%m-%d")
end_date = datetime.strptime("2020-06-30", "%Y-%m-%d")


t = 1
while start_date < end_date:
    suffix = f'_t{t}'
    BANDS_suffixes =[band + suffix for band in BANDS]

    tr_irn[BANDS_suffixes] = None
    te_irn[BANDS_suffixes] = None

    current_date = start_date
    next_date = start_date + timedelta(days=30)
    print('############', current_date, ' to ', next_date)
    collection = (ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))


    #Extract train data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(tr_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(tr_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        tr_irn.loc[i, BANDS_suffixes] = values
        i+=1


     #Extract test data - the mean values for the specified bands
    mean_values = (collection.select(BANDS).filterBounds(ee.FeatureCollection(te_point_geometries)).mean().
                   reduceRegions(collection=ee.FeatureCollection(te_point_geometries), reducer=ee.Reducer.mean(), scale=10)
                  )

    i = 0
    for feature in tqdm(mean_values.getInfo()['features']):
        values = [feature['properties'][band] for band in BANDS]
        te_irn.loc[i, BANDS_suffixes] = values
        i+=1

    start_date = next_date
    t+=1

############ 2019-07-01 00:00:00  to  2019-07-31 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1957.41it/s]


############ 2019-07-31 00:00:00  to  2019-08-30 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 2050.39it/s]


############ 2019-08-30 00:00:00  to  2019-09-29 00:00:00


...

############ 2020-05-26 00:00:00  to  2020-06-25 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1605.67it/s]


############ 2020-06-25 00:00:00  to  2020-07-25 00:00:00


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 2021.21it/s]

CPU times: user 8.88 s, sys: 134 ms, total: 9.01 s
Wall time: 3min 4s


In [21]:
tr_afg.to_csv(f'tr_afg_s1.csv', index=False )
te_afg.to_csv(f'te_afg_s1.csv', index=False)

tr_sud.to_csv(f'tr_sud_s1.csv', index=False)
te_sud.to_csv(f'te_sud_s1.csv', index=False)

tr_irn.to_csv(f'tr_irn_s1.csv', index=False)
te_irn.to_csv(f'te_irn_s1.csv', index=False)

## Modeling

In [24]:
#!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.1 MB/s eta 0:00:00


In [22]:
# Sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Machine Learning
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping, log_evaluation
from catboost import CatBoostClassifier

In [23]:
## Function to compute different available vegitation indices from sentinel-2
def s2_veg_indices(df, t):
    veg_df = pd.DataFrame()
    veg_df[f'MNDVI_t{t}'] = (df[f'B8_t{t}'] - df[f'B4_t{t}'] )  / (df[f'B8_t{t}'] +df[f'B4_t{t}'] - 2*df[f'B2_t{t}'])
    veg_df[f'NDSI_t{t}'] =  df[f'B3_t{t}']  / (df[f'B11_t{t}'])
    veg_df[f'NDVI_t{t}'] =  (df[f'B8_t{t}'] - df[f'B4_t{t}'] )  / (df[f'B8_t{t}'] +df[f'B4_t{t}'] )
    veg_df[f'NDWI_t{t}'] = (df[f'B3_t{t}'] -  df[f'B8_t{t}'] )  / (df[f'B3_t{t}'] +df[f'B8_t{t}'])
    veg_df[f'NDMI_t{t}'] = (df[f'B8_t{t}'] - df[f'B11_t{t}'] )  / (df[f'B8_t{t}'] +df[f'B11_t{t}' ])
    veg_df[f'NDBI_t{t}'] = (df[f'B11_t{t}'] - df[f'B8_t{t}'] )  / (df[f'B11_t{t}'] +df[f'B8_t{t}'])
    veg_df[f'NDCI_t{t}'] = (df[f'B5_t{t}'] - df[f'B4_t{t}'] )  / (df[f'B5_t{t}'] +df[f'B4_t{t}'])
    veg_df[f'GNDVI_t{t}'] = (df[f'B8_t{t}'] - df[f'B3_t{t}'] )  / (df[f'B8_t{t}'] +df[f'B3_t{t}'] )


    veg_df[f'BSI_t{t}'] =  (df[f'B11_t{t}'] - df[f'B4_t{t}'] )  / (df[f'B8_t{t}'] +df[f'B2_t{t}'])
    veg_df[f'NDVI_R_t{t}'] =  (df[f'B8_t{t}'] - df[f'B7_t{t}'] )  / (df[f'B8_t{t}'] +df[f'B7_t{t}'])
    veg_df[f'CHL_t{t}'] =  (df[f'B7_t{t}'] / (df[f'B5_t{t}']))  - 1
    veg_df[f'CVI_t{t}'] =  (df[f'B8_t{t}'] / (df[f'B3_t{t}']))  * (df[f'B4_t{t}'] / (df[f'B3_t{t}']))
    veg_df[f'BI_t{t}'] =  (df[f'B4_t{t}'] **2+ df[f'B3_t{t}']**2+ df[f'B2_t{t}']*2) /3
    veg_df[f'SI_t{t}'] =  (df[f'B4_t{t}'] - df[f'B2_t{t}'])  / (df[f'B4_t{t}'] + df[f'B2_t{t}'])
    veg_df[f'NMDI_t{t}'] =  (df[f'B8_t{t}'] - (df[f'B11_t{t}'] - df[f'B12_t{t}']))  / (df[f'B8_t{t}'] + (df[f'B11_t{t}'] - df[f'B12_t{t}']))
    veg_df[f'MSI_t{t}'] =  df[f'B11_t{t}']  / (df[f'B8_t{t}'])
    veg_df[f'BSI1_t{t}'] =  ((df[f'B12_t{t}'] + df[f'B4_t{t}']) - (df[f'B8A_t{t}'] + df[f'B2_t{t}']))  / ((df[f'B12_t{t}'] + df[f'B4_t{t}']) + (df[f'B8A_t{t}'] + df[f'B2_t{t}']))
    veg_df[f'BSI2_t{t}'] =  ((df[f'B11_t{t}'] + df[f'B4_t{t}']) - (df[f'B8A_t{t}'] + df[f'B2_t{t}']))  / ((df[f'B11_t{t}'] + df[f'B4_t{t}']) + (df[f'B8A_t{t}'] + df[f'B2_t{t}']))
    veg_df[f'NDSI1_t{t}'] = (df[f'B11_t{t}'] - df[f'B8A_t{t}'] )  / (df[f'B11_t{t}'] + df[f'B8A_t{t}'] )
    veg_df[f'NDSI2_t{t}'] = (df[f'B12_t{t}'] - df[f'B3_t{t}'] )  / (df[f'B12_t{t}'] + df[f'B3_t{t}'] )
    veg_df[f'BI2_t{t}'] = df[f'B4_t{t}'] + df[f'B11_t{t}'] - df[f'B8A_t{t}']
    veg_df[f'DBSI_t{t}'] = veg_df[f'NDSI2_t{t}'] - ((df[f'B8A_t{t}'] - df[f'B4_t{t}']) / (df[f'B8A_t{t}'] + df[f'B4_t{t}']))
    veg_df[f'MBI_t{t}'] = ((df[f'B11_t{t}'] + df[f'B12_t{t}'] + df[f'B8A_t{t}']) / (df[f'B11_t{t}'] + df[f'B12_t{t}'] + df[f'B8A_t{t}'])) + 0.5
    veg_df[f'R03_t{t}'] =  df[f'B11_t{t}']  / (df[f'B12_t{t}'])
    veg_df[f'R04_t{t}'] =  df[f'B5_t{t}']  / (df[f'B4_t{t}'])
    veg_df[f'MI_t{t}'] = (df[f'B8A_t{t}'] - df[f'B11_t{t}'] )  / (df[f'B8A_t{t}'] + df[f'B11_t{t}'] )
    veg_df[f'PSRI_t{t}'] = (df[f'B4_t{t}'] - df[f'B2_t{t}'] )  / (df[f'B6_t{t}'] )
    veg_df[f'TVI_t{t}'] = (120*(df[f'B6_t{t}'] - df[f'B3_t{t}'] ) - 200 * (df[f'B4_t{t}'] - df[f'B3_t{t}'])) / 2
    veg_df[f'ARVI_t{t}'] = (df[f'B8_t{t}'] - 2*df[f'B4_t{t}'] + df[f'B2_t{t}'])  / (df[f'B8_t{t}'] + 2*df[f'B4_t{t}'] + df[f'B2_t{t}'])
    veg_df[f'SIPI_t{t}'] =  (df[f'B8_t{t}'] - df[f'B2_t{t}'] )  / (df[f'B8_t{t}'] +df[f'B4_t{t}'] )
    veg_df[f'EXG_t{t}'] = (2 * df[f'B3_t{t}'] -  df[f'B4_t{t}'] -  df[f'B2_t{t}'] )
    veg_df[f'ACI_t{t}'] = (df[f'B8_t{t}']  )  * (df[f'B4_t{t}'] +df[f'B3_t{t}'] )

    # REDEDGE indices
    veg_df[f'NDVIre1_t{t}'] =  (df[f'B8_t{t}'] - df[f'B5_t{t}'])  / (df[f'B8_t{t}'] + df[f'B5_t{t}'])
    veg_df[f'NDVIre2_t{t}'] =  (df[f'B8_t{t}'] - df[f'B6_t{t}'])  / (df[f'B8_t{t}'] + df[f'B6_t{t}'])
    veg_df[f'NDVIre3_t{t}'] =  (df[f'B8_t{t}'] - df[f'B7_t{t}'])  / (df[f'B8_t{t}'] + df[f'B7_t{t}'])

    veg_df[f'NDRE1_t{t}'] =  (df[f'B6_t{t}'] - df[f'B5_t{t}'])  / (df[f'B6_t{t}'] + df[f'B5_t{t}'])
    veg_df[f'NDRE2_t{t}'] =  (df[f'B7_t{t}'] - df[f'B5_t{t}'])  / (df[f'B7_t{t}'] + df[f'B5_t{t}'])
    veg_df[f'NDRE3_t{t}'] =  (df[f'B7_t{t}'] - df[f'B6_t{t}'])  / (df[f'B7_t{t}'] + df[f'B6_t{t}'])

    veg_df[f'CIre1_t{t}'] =  (df[f'B8_t{t}'] /(df[f'B5_t{t}']))  - 1
    veg_df[f'CIre2_t{t}'] =  (df[f'B8_t{t}'] /(df[f'B6_t{t}']))  - 1
    veg_df[f'CIre3_t{t}'] =  (df[f'B8_t{t}'] /(df[f'B7_t{t}']))  - 1

    veg_df[f'MCARI1_t{t}'] =  ((df[f'B5_t{t}'] - df[f'B4_t{t}']) - 0.2*(df[f'B5_t{t}'] - df[f'B3_t{t}'])) * (df[f'B5_t{t}'] / (df[f'B4_t{t}']))
    veg_df[f'MCARI2_t{t}'] =  ((df[f'B6_t{t}'] - df[f'B4_t{t}']) - 0.2*(df[f'B6_t{t}'] - df[f'B3_t{t}'])) * (df[f'B6_t{t}'] / (df[f'B4_t{t}']))
    veg_df[f'MCARI3_t{t}'] =  ((df[f'B7_t{t}'] - df[f'B4_t{t}']) - 0.2*(df[f'B7_t{t}'] - df[f'B3_t{t}'])) * (df[f'B7_t{t}'] / (df[f'B4_t{t}']))


    veg_df[f'TCARI1_t{t}'] =  3*((df[f'B5_t{t}'] - df[f'B4_t{t}']) - 0.2*(df[f'B5_t{t}'] - df[f'B3_t{t}'])) * (df[f'B5_t{t}'] / (df[f'B4_t{t}']))
    veg_df[f'TCARI2_t{t}'] =  3*((df[f'B6_t{t}'] - df[f'B4_t{t}']) - 0.2*(df[f'B6_t{t}'] - df[f'B3_t{t}'])) * (df[f'B6_t{t}'] / (df[f'B4_t{t}']))
    veg_df[f'TCARI3_t{t}'] =  3*((df[f'B7_t{t}'] - df[f'B4_t{t}']) - 0.2*(df[f'B7_t{t}'] - df[f'B3_t{t}'])) * (df[f'B7_t{t}'] / (df[f'B4_t{t}']))

    veg_df[f'MTCI1_t{t}'] =  (df[f'B6_t{t}'] - df[f'B5_t{t}'])  / (df[f'B5_t{t}'] - df[f'B4_t{t}'] + 1e-6)
    veg_df[f'MTCI2_t{t}'] =  (df[f'B7_t{t}'] - df[f'B5_t{t}'])  / (df[f'B5_t{t}'] - df[f'B4_t{t}']+ 1e-6)
    veg_df[f'MTCI3_t{t}'] =  (df[f'B7_t{t}'] - df[f'B6_t{t}'])  / (df[f'B6_t{t}'] - df[f'B4_t{t}']+ 1e-6)

    # Blooming Indices (to detect flowers colors (purple, yellow) of different crops)
    veg_df[f'NDGI_t{t}'] =  (df[f'B4_t{t}'] - df[f'B3_t{t}'] )  / (df[f'B4_t{t}'] +df[f'B3_t{t}'] )
    veg_df[f'DYI_t{t}'] =  df[f'B4_t{t}']  / df[f'B3_t{t}']
    veg_df[f'NDPI_t{t}'] =  (0.5*(df[f'B4_t{t}'] + df[f'B2_t{t}']) - df[f'B3_t{t}'])  / (0.5*(df[f'B4_t{t}'] + df[f'B2_t{t}']) + df[f'B3_t{t}'])
    veg_df[f'PEBI_t{t}'] =  veg_df[f'NDPI_t{t}'] / ((veg_df[f'NDGI_t{t}'] +1) * df[f'B8_t{t}'])
    veg_df[f'NDYI_t{t}'] =  (0.5*(df[f'B4_t{t}'] + df[f'B3_t{t}']) - df[f'B2_t{t}'])  / (0.5*(df[f'B4_t{t}'] + df[f'B3_t{t}']) + df[f'B2_t{t}'])
    veg_df[f'YEBI_t{t}'] =  veg_df[f'NDYI_t{t}'] / ((veg_df[f'NDGI_t{t}'] +1) * df[f'B8_t{t}'])


    veg_df['ID'] = list(df['ID'])

    return veg_df

In [24]:
## Function to compute different available vegitation indices from sentinel-1
def s1_veg_indices(df, t):
    veg_df = pd.DataFrame()

    veg_df[f'MNDVI_t{t}'] = df[f'VV_t{t}'] /df[f'VH_t{t}']
    RVI = (4*df[f'VH_t{t}']) / (df[f'VV_t{t}'] + df[f'VH_t{t}'])
    veg_df[f'RVI_t{t}'] =  list(RVI)

    veg_df[f'VV*VH_t{t}'] = df[f'VV_t{t}'] * df[f'VH_t{t}']

    DOP = df[f'VV_t{t}'] / (df[f'VV_t{t}'] + df[f'VH_t{t}'])
    RVI4 = list(np.sqrt(DOP) * RVI)
    veg_df[f'RVI4_t{t}'] = list(RVI4)


    veg_df['ID'] = list(df['ID'])
    return veg_df

### Afghanistan

In [25]:
# Read teh extractd data from the 3 different sources

## Sentinel-2
tr_afg_s2 = pd.read_csv(f'tr_afg_s2.csv')
te_afg_s2 = pd.read_csv(f'te_afg_s2.csv')

## Sentinel-1
tr_afg_s1 = pd.read_csv(f'tr_afg_s1.csv')
te_afg_s1 = pd.read_csv(f'te_afg_s1.csv')

## SMAP
tr_afg_smap = pd.read_csv(f'tr_afg_smap.csv')
te_afg_smap = pd.read_csv(f'te_afg_smap.csv')

In [26]:
# Compute the sentinel-2 vegitation indecies for the 4 months
for time in range(4):
    tr_veg_indices = s2_veg_indices(tr_afg_s2, t= time+1)
    tr_afg_s2 = pd.merge(tr_afg_s2, tr_veg_indices, on=['ID'], how='inner')

    te_veg_indices = s2_veg_indices(te_afg_s2, t= time+1)
    te_afg_s2 = pd.merge(te_afg_s2, te_veg_indices, on=['ID'], how='inner')


# Compute the sentinel-1 vegitation indecies for the 4 months
for time in range(4):
    tr_veg_indices = s1_veg_indices(tr_afg_s1, t= time+1)
    tr_afg_s1 = pd.merge(tr_afg_s1, tr_veg_indices, on=['ID'], how='inner')

    te_veg_indices = s1_veg_indices(te_afg_s1, t= time+1)
    te_afg_s1 = pd.merge(te_afg_s1, te_veg_indices, on=['ID'], how='inner')

In [27]:
# Merge the computed vegetation indecies and the other extracted data into 1 dataset
tr_afg = pd.merge(tr_afg_s2, tr_afg_s1, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster','Target'], how='inner')
tr_afg = pd.merge(tr_afg, tr_afg_smap, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster','Target'], how='inner')


te_afg = pd.merge(te_afg_s2, te_afg_s1, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster'], how='inner')
te_afg = pd.merge(te_afg, te_afg_smap, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster'], how='inner')

In [28]:
X = tr_afg.drop(['ID', 'Target', 'geometry', 'Cluster', 'Lat', 'Lon'], axis = 1).fillna(-99999)
y = tr_afg.Target
test_df = te_afg.drop(['ID', 'geometry', 'Cluster' , 'Lat', 'Lon'], axis = 1).fillna(-99999)

print(X.shape)
print(test_df.shape)
print(y.shape)

(500, 320)
(500, 320)
(500,)


In [29]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=SEED) # for cross validation
afg_lgbmscores = []
afg_lgbmpreds= []


# Creating loop for the stratified k fold
i = 0
for train, val in skf.split(X, y):
    print(f'########### Fold number {i+1} ')

    # spliting the data
    x_train, x_val = X.iloc[train], X.iloc[val]
    y_train, y_val = y.iloc[train], y.iloc[val]

    clf = LGBMClassifier(boosting_type='gbdt',learning_rate=0.08,
                           n_estimators=2000,deterministic=True, objective='binary',
                           subsample=0.90, subsample_freq=5,
                           random_state=SEED,n_jobs=- 1)

    # fitting on train data
    clf.fit( x_train, y_train, eval_set = (x_val,y_val), callbacks=[log_evaluation(period=150), early_stopping(200)])

    # Making predictions
    y_pred = clf.predict(x_val)

    # Measuring the accuracy of the model
    score = accuracy_score(y_val, y_pred)
    print(f'Accuracy Score: {score}')
    afg_lgbmscores.append(score)

    preds = clf.predict_proba(test_df)
    afg_lgbmpreds.append(preds)
    i+=1


print(f'Mean accuracy: {np.mean(afg_lgbmscores)}')

########### Fold number 1 
Training until validation scores don't improve for 200 rounds
[150]	valid_0's binary_logloss: 0.443621
Early stopping, best iteration is:
[41]	valid_0's binary_logloss: 0.337541
Accuracy Score: 0.87
########### Fold number 2 
Training until validation scores don't improve for 200 rounds
[150]	valid_0's binary_logloss: 0.628191
Early stopping, best iteration is:
[32]	valid_0's binary_logloss: 0.364412
Accuracy Score: 0.86
########### Fold number 3 
Training until validation scores don't improve for 200 rounds
[150]	valid_0's binary_logloss: 0.494832
Early stopping, best iteration is:
[66]	valid_0's binary_logloss: 0.370716
Accuracy Score: 0.84
########### Fold number 4 
Training until validation scores don't improve for 200 rounds
[150]	valid_0's binary_logloss: 0.606033
Early stopping, best iteration is:
[40]	valid_0's binary_logloss: 0.381801
Accuracy Score: 0.84
########### Fold number 5 
Training until validation scores don't improve for 200 rounds
[150]	v

In [30]:
afg_lgbmpreds_mean = np.mean(afg_lgbmpreds, axis=0)

In [31]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=SEED) # for cross validation
afg_catscores = []
afg_catpreds= []


# Creating loop for the stratified k fold
i = 0
for train, val in skf.split(X, y):
    print(f'########### Fold number {i+1} ')

    # spliting the data
    x_train, x_val = X.iloc[train], X.iloc[val]
    y_train, y_val = y.iloc[train], y.iloc[val]

    clf = CatBoostClassifier(iterations=30000,  has_time=True ,bootstrap_type='No',random_strength=0,
                                   learning_rate=0.05,use_best_model=True,
                                   random_seed=SEED)
    # fitting on train data
    clf.fit( x_train, y_train, eval_set = (x_val,y_val),verbose=500 ,early_stopping_rounds=300)

    # Making predictions
    y_pred = clf.predict(x_val)

    # Measuring the accuracy of the model
    score = accuracy_score(y_val, y_pred)
    print(f'Accuracy Score: {score}')
    afg_catscores.append(score)

    preds = clf.predict_proba(test_df)
    afg_catpreds.append(preds)
    i+=1


print(f'Mean accuracy: {np.mean(afg_catscores)}')

########### Fold number 1 
0:	learn: 0.6395486	test: 0.6563229	best: 0.6563229 (0)	total: 55.7ms	remaining: 27m 52s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.3469863027
bestIteration = 99

Shrink model to first 100 iterations.
Accuracy Score: 0.85
########### Fold number 2 
0:	learn: 0.6351133	test: 0.6426304	best: 0.6426304 (0)	total: 13.1ms	remaining: 6m 34s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.3712185627
bestIteration = 27

Shrink model to first 28 iterations.
Accuracy Score: 0.86
########### Fold number 3 
0:	learn: 0.6411887	test: 0.6552670	best: 0.6552670 (0)	total: 6.73ms	remaining: 3m 21s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.3303030595
bestIteration = 80

Shrink model to first 81 iterations.
Accuracy Score: 0.86
########### Fold number 4 
0:	learn: 0.6385482	test: 0.6623217	best: 0.6623217 (0)	total: 7.18ms	remaining: 3m 35s
Stopped by overfitting detector  (300 iterations wait)

bestTest

In [32]:
afg_catpreds_mean = np.mean(afg_catpreds, axis=0)

In [33]:
blend_afg = 0.40*afg_catpreds_mean + 0.60*afg_lgbmpreds_mean

blend_afg = np.argmax(blend_afg, axis=1)
blend_afg = pd.DataFrame({'ID': te_afg.ID, 'Target': blend_afg})
blend_afg.head()

,ID,Target
0,ID_9ZLHTVF6NSU7,1
1,ID_B2WO8GOJOMY1,0
2,ID_K82JJ5PQCMXM,1
3,ID_5LACW9CE2OIB,1
4,ID_O6RIQWT1103E,0


### Sudan

In [34]:
# Read teh extractd data from the 3 different sources

## Sentinel-2
tr_sud_s2 = pd.read_csv(f'tr_sud_s2.csv')
te_sud_s2 = pd.read_csv(f'te_sud_s2.csv')

## Sentinel-1
tr_sud_s1 = pd.read_csv(f'tr_sud_s1.csv')
te_sud_s1 = pd.read_csv(f'te_sud_s1.csv')

## SMAP
tr_sud_smap = pd.read_csv(f'tr_sud_smap.csv')
te_sud_smap = pd.read_csv(f'te_sud_smap.csv')

In [35]:
# Compute the sentinel-2 vegitation indecies for the entire timeseries
for time in range(13):
    tr_veg_indices = s2_veg_indices(tr_sud_s2, t= time+1)
    tr_sud_s2 = pd.merge(tr_sud_s2, tr_veg_indices, on=['ID'], how='inner')

    te_veg_indices = s2_veg_indices(te_sud_s2, t= time+1)
    te_sud_s2 = pd.merge(te_sud_s2, te_veg_indices, on=['ID'], how='inner')


# Compute the sentinel-1 vegitation indecies for the entire timeseries
for time in range(13):
    tr_veg_indices = s1_veg_indices(tr_sud_s1, t= time+1)
    tr_sud_s1 = pd.merge(tr_sud_s1, tr_veg_indices, on=['ID'], how='inner')

    te_veg_indices = s1_veg_indices(te_sud_s1, t= time+1)
    te_sud_s1 = pd.merge(te_sud_s1, te_veg_indices, on=['ID'], how='inner')

/opt/conda/envs/eurac_env/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/envs/eurac_env/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/envs/eurac_env/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [36]:
# Merge the computed vegetation indecies and the other extracted data into 1 dataset
tr_sud = pd.merge(tr_sud_s2, tr_sud_s1, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster','Target'], how='inner')
tr_sud = pd.merge(tr_sud, tr_sud_smap, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster','Target'], how='inner')

te_sud = pd.merge(te_sud_s2, te_sud_s1, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster'], how='inner')
te_sud = pd.merge(te_sud, te_sud_smap, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster'], how='inner')

In [37]:
X = tr_sud.drop(['ID', 'Target', 'geometry', 'Cluster', 'Lat', 'Lon'], axis = 1).fillna(-99999)
y = tr_sud.Target
test_df = te_sud.drop(['ID', 'geometry', 'Cluster', 'Lat', 'Lon' ], axis = 1).fillna(-99999)

print(X.shape)
print(test_df.shape)
print(y.shape)

(500, 1040)
(500, 1040)
(500,)


In [38]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=SEED) # for cross validation
sud_lgbmscores = []
sud_lgbmpreds= []


# Creating loop for the stratified k fold
i = 0
for train, val in skf.split(X, y):
    print(f'########### Fold number {i+1} ')

    # spliting the data
    x_train, x_val = X.iloc[train], X.iloc[val]
    y_train, y_val = y.iloc[train], y.iloc[val]

    clf = LGBMClassifier(boosting_type='gbdt',learning_rate=0.05, #0.05
                         n_estimators=2000,deterministic=True, objective='binary',
                         subsample=0.90, subsample_freq=5,
                         random_state=SEED,n_jobs=- 1)

    # fitting on train data
    clf.fit( x_train, y_train, eval_set = (x_val,y_val),
            callbacks=[log_evaluation(period=150), early_stopping(300)])

    # Making predictions
    y_pred = clf.predict(x_val)

    # Measuring the accuracy of the model
    score = accuracy_score(y_val, y_pred)
    print(f'Accuracy Score: {score}')
    sud_lgbmscores.append(score)

    preds = clf.predict_proba(test_df)
    sud_lgbmpreds.append(preds)
    i+=1


print(f'Mean accuracy: {np.mean(sud_lgbmscores)}')

########### Fold number 1 
Training until validation scores don't improve for 300 rounds
[150]	valid_0's binary_logloss: 0.131538
[300]	valid_0's binary_logloss: 0.122632
[450]	valid_0's binary_logloss: 0.124889
Early stopping, best iteration is:
[264]	valid_0's binary_logloss: 0.110848
Accuracy Score: 0.96
########### Fold number 2 
Training until validation scores don't improve for 300 rounds
[150]	valid_0's binary_logloss: 0.100499
[300]	valid_0's binary_logloss: 0.0742387
[450]	valid_0's binary_logloss: 0.0781716
Early stopping, best iteration is:
[274]	valid_0's binary_logloss: 0.0640944
Accuracy Score: 0.97
########### Fold number 3 
Training until validation scores don't improve for 300 rounds
[150]	valid_0's binary_logloss: 0.0958398
[300]	valid_0's binary_logloss: 0.106639
Early stopping, best iteration is:
[130]	valid_0's binary_logloss: 0.0884784
Accuracy Score: 0.96
########### Fold number 4 
Training until validation scores don't improve for 300 rounds
[150]	valid_0's bina

In [39]:
sud_lgbmpreds_mean = np.mean(sud_lgbmpreds, axis=0)

In [40]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=SEED) # for cross validation
sud_catscores = []
sud_catpreds= []


# Creating loop for the stratified k fold
i = 0
for train, val in skf.split(X, y):
    print(f'########### Fold number {i+1} ')

    # spliting the data
    x_train, x_val = X.iloc[train], X.iloc[val]
    y_train, y_val = y.iloc[train], y.iloc[val]

    clf = CatBoostClassifier(iterations=30000,  has_time=True ,bootstrap_type='No',
                             random_strength=0,
                             learning_rate=0.01,use_best_model=True,#0.08
                             random_seed=SEED)

    # fitting on train data
    clf.fit( x_train, y_train, eval_set = (x_val,y_val),verbose=500 ,early_stopping_rounds=300)

    # Making predictions
    y_pred = clf.predict(x_val)

    # Measuring the accuracy of the model
    score = accuracy_score(y_val, y_pred)
    print(f'Accuracy Score: {score}')
    sud_catscores.append(score)

    preds = clf.predict_proba(test_df)
    sud_catpreds.append(preds)
    i+=1


print(f'Mean accuracy: {np.mean(sud_catscores)}')

########### Fold number 1 
0:	learn: 0.6742880	test: 0.6785644	best: 0.6785644 (0)	total: 22.1ms	remaining: 11m 1s
500:	learn: 0.0059129	test: 0.1276669	best: 0.1243471 (375)	total: 11.1s	remaining: 10m 53s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.1243470954
bestIteration = 375

Shrink model to first 376 iterations.
Accuracy Score: 0.95
########### Fold number 2 
0:	learn: 0.6763387	test: 0.6782827	best: 0.6782827 (0)	total: 20.1ms	remaining: 10m 1s
500:	learn: 0.0051574	test: 0.0954140	best: 0.0954140 (500)	total: 11s	remaining: 10m 50s
1000:	learn: 0.0023337	test: 0.0884422	best: 0.0884422 (1000)	total: 21.6s	remaining: 10m 25s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.08750726767
bestIteration = 1085

Shrink model to first 1086 iterations.
Accuracy Score: 0.95
########### Fold number 3 
0:	learn: 0.6756061	test: 0.6774059	best: 0.6774059 (0)	total: 32.1ms	remaining: 16m 4s
500:	learn: 0.0060991	test: 0.1161138	best: 0.1068479 (2

In [41]:
sud_catpreds_mean = np.mean(sud_catpreds, axis=0)

In [42]:
blend_sud = 0.55*sud_lgbmpreds_mean + 0.45*sud_catpreds_mean

blend_sud = np.argmax(blend_sud, axis=1)
blend_sud = pd.DataFrame({'ID': te_sud.ID, 'Target': blend_sud})
blend_sud.head()

,ID,Target
0,ID_SOYSG7W04UH3,1
1,ID_EAP7EXXV8ZDE,1
2,ID_QPRX1TUQVGHU,0
3,ID_C78YQ32G1KO9,0
4,ID_M5X39UIEM64N,1


### Iran

In [43]:
# Read teh extractd data from the 3 different sources

## Sentinel-2
tr_irn_s2 = pd.read_csv(f'tr_irn_s2.csv')
te_irn_s2 = pd.read_csv(f'te_irn_s2.csv')

## Sentinel-1
tr_irn_s1 = pd.read_csv(f'tr_irn_s1.csv')
te_irn_s1 = pd.read_csv(f'te_irn_s1.csv')

## SMAP
tr_irn_smap = pd.read_csv(f'tr_irn_smap.csv')
te_irn_smap = pd.read_csv(f'te_irn_smap.csv')

In [44]:
# Compute the sentinel-2 vegitation indecies for the entire timeseries
for time in range(13):
    tr_veg_indices = s2_veg_indices(tr_irn_s2, t= time+1)
    tr_irn_s2 = pd.merge(tr_irn_s2, tr_veg_indices, on=['ID'], how='inner')

    te_veg_indices = s2_veg_indices(te_irn_s2, t= time+1)
    te_irn_s2 = pd.merge(te_irn_s2, te_veg_indices, on=['ID'], how='inner')


# Compute the sentinel-1 vegitation indecies for the entire timeseries
for time in range(13):
    tr_veg_indices = s1_veg_indices(tr_irn_s1, t= time+1)
    tr_irn_s1 = pd.merge(tr_irn_s1, tr_veg_indices, on=['ID'], how='inner')

    te_veg_indices = s1_veg_indices(te_irn_s1, t= time+1)
    te_irn_s1 = pd.merge(te_irn_s1, te_veg_indices, on=['ID'], how='inner')

/opt/conda/envs/eurac_env/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/envs/eurac_env/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/envs/eurac_env/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/envs/eurac_env/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/envs/eurac_env/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/envs/eurac_env/lib/python3.8/site-packages/pandas/core

In [45]:
# Merge the computed vegetation indecies and the other extracted data into 1 dataset
tr_irn = pd.merge(tr_irn_s2, tr_irn_s1, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster','Target'], how='inner')
tr_irn = pd.merge(tr_irn, tr_irn_smap, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster','Target'], how='inner')

te_irn = pd.merge(te_irn_s2, te_irn_s1, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster'], how='inner')
te_irn = pd.merge(te_irn, te_irn_smap, on=['ID', 'Lat', 'Lon', 'geometry', 'Cluster'], how='inner')

In [46]:
X = tr_irn.drop(['ID', 'Target', 'geometry', 'Cluster', 'Lat', 'Lon'], axis = 1).fillna(-99999)
y = tr_irn.Target
test_df = te_irn.drop(['ID', 'geometry', 'Cluster', 'Lat', 'Lon' ], axis = 1).fillna(-99999)

print(X.shape)
print(test_df.shape)
print(y.shape)

(500, 1040)
(500, 1040)
(500,)


In [47]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=SEED) # for cross validation
irn_lgbmscores = []
irn_lgbmpreds= []


# Creating loop for the stratified k fold
i = 0
for train, val in skf.split(X, y):
    print(f'########### Fold number {i+1} ')

    # spliting the data
    x_train, x_val = X.iloc[train], X.iloc[val]
    y_train, y_val = y.iloc[train], y.iloc[val]

    clf = LGBMClassifier(boosting_type='gbdt',learning_rate=0.05,#0.08
                         n_estimators=2000,deterministic=True, objective='binary',
                         subsample=0.90, subsample_freq=10,
                         random_state=SEED,n_jobs=- 1)

    # fitting on train data
    clf.fit( x_train, y_train, eval_set = (x_val,y_val), callbacks=[log_evaluation(period=150), early_stopping(200)])

    # Making predictions
    y_pred = clf.predict(x_val)

    # Measuring the accuracy of the model
    score = accuracy_score(y_val, y_pred)
    print(f'Accuracy Score: {score}')
    irn_lgbmscores.append(score)

    preds = clf.predict_proba(test_df)
    irn_lgbmpreds.append(preds)
    i+=1


print(f'Mean accuracy: {np.mean(irn_lgbmscores)}')

########### Fold number 1 
Training until validation scores don't improve for 200 rounds
[150]	valid_0's binary_logloss: 0.0807749
[300]	valid_0's binary_logloss: 0.11461
Early stopping, best iteration is:
[194]	valid_0's binary_logloss: 0.0732114
Accuracy Score: 0.97
########### Fold number 2 
Training until validation scores don't improve for 200 rounds
[150]	valid_0's binary_logloss: 0.10424
[300]	valid_0's binary_logloss: 0.140228
Early stopping, best iteration is:
[125]	valid_0's binary_logloss: 0.0984734
Accuracy Score: 0.99
########### Fold number 3 
Training until validation scores don't improve for 200 rounds
[150]	valid_0's binary_logloss: 0.220543
Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.174141
Accuracy Score: 0.96
########### Fold number 4 
Training until validation scores don't improve for 200 rounds
[150]	valid_0's binary_logloss: 0.103944
[300]	valid_0's binary_logloss: 0.118805
Early stopping, best iteration is:
[205]	valid_0's binary_logloss:

In [48]:
irn_lgbmpreds_mean = np.mean(irn_lgbmpreds, axis=0)

In [49]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=SEED) # for cross validation
irn_catscores = []
irn_catpreds= []


# Creating loop for the stratified k fold
i = 0
for train, val in skf.split(X, y):
    print(f'########### Fold number {i+1} ')

    # spliting the data
    x_train, x_val = X.iloc[train], X.iloc[val]
    y_train, y_val = y.iloc[train], y.iloc[val]

    clf = CatBoostClassifier(iterations=30000,  has_time=True ,bootstrap_type='No',random_strength=0,
                                   learning_rate=0.08,use_best_model=True,
                                   random_seed=SEED)
    # fitting on train data
    clf.fit( x_train, y_train, eval_set = (x_val,y_val),verbose=500 ,early_stopping_rounds=300)

    # Making predictions
    y_pred = clf.predict(x_val)

    # Measuring the accuracy of the model
    score = accuracy_score(y_val, y_pred)
    print(f'Accuracy Score: {score}')
    irn_catscores.append(score)

    preds = clf.predict_proba(test_df)
    irn_catpreds.append(preds)
    i+=1


print(f'Mean accuracy: {np.mean(irn_catscores)}')

########### Fold number 1 
0:	learn: 0.5517591	test: 0.5633655	best: 0.5633655 (0)	total: 21.1ms	remaining: 10m 32s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.0905574612
bestIteration = 90

Shrink model to first 91 iterations.
Accuracy Score: 0.97
########### Fold number 2 
0:	learn: 0.5449474	test: 0.5635683	best: 0.5635683 (0)	total: 22.2ms	remaining: 11m 4s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.1144334504
bestIteration = 110

Shrink model to first 111 iterations.
Accuracy Score: 0.95
########### Fold number 3 
0:	learn: 0.5490407	test: 0.5766901	best: 0.5766901 (0)	total: 21.4ms	remaining: 10m 41s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.2016859296
bestIteration = 42

Shrink model to first 43 iterations.
Accuracy Score: 0.95
########### Fold number 4 
0:	learn: 0.5439856	test: 0.5726367	best: 0.5726367 (0)	total: 39.1ms	remaining: 19m 34s
Stopped by overfitting detector  (300 iterations wait)

bestT

In [50]:
irn_catpreds_mean = np.mean(irn_catpreds, axis=0)

In [51]:
blend_irn = 0.60*irn_lgbmpreds_mean + 0.40*irn_catpreds_mean


blend_irn = np.argmax(blend_irn, axis=1)
blend_irn = pd.DataFrame({'ID': te_irn.ID, 'Target': blend_irn})
blend_irn.head()

,ID,Target
0,ID_LNN7BFCVEZKA,0
1,ID_ZMB4I2ZXYE4X,1
2,ID_OFRXD08BLP3X,0
3,ID_IQ4IS9AL13PV,1
4,ID_X7ZL15DE59SA,0


### Merge the regional prediction into 1 dataframe

In [52]:
sub_file = pd.concat([blend_irn, blend_afg, blend_sud]) .reset_index(drop=True)

In [53]:
sample_submission = sample_submission.drop(columns='Target')
sample_submission = pd.merge(sample_submission, sub_file, on=['ID'], how='inner')
sample_submission.to_csv('predictions.csv', index=False)